In [2]:
from cnn_model_3 import EmotionRecognition

In [3]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
train_loader = torch.load('train_dataloader.pth')
test_loader = torch.load('test_dataloader.pth')

Using device: cuda


C:\Users\Korisnik\AppData\Local\Temp\ipykernel_32216\3366855997.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loader = torch.load('train_dataloader.pth')
C:\User

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmotionRecognition(nn.Module):
    def __init__(self):
        super(EmotionRecognition, self).__init__()

        # Convolutional Block 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # Convolutional Block 2
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # Dummy input to determine flatten size after conv layers
        with torch.no_grad():
            dummy = torch.zeros(1, 3, 48, 48)
            dummy = self._forward_conv(dummy)
            self.flattened_size = dummy.view(1, -1).size(1)

        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc2 = nn.Linear(512, 7)

    def _forward_conv(self, x):
        # Only the convolutional part, used for shape inference and forward
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [5]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        alpha: tensor of shape (num_classes,) for class weights
        gamma: focusing parameter
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # class weights
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)  # pt = probability of the true class
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [7]:
import torch
import torch.optim as optim
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
train_loader = torch.load('train_dataloader.pth')
test_loader = torch.load('test_dataloader.pth')
model = EmotionRecognition().to(device)
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn

class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_loader.dataset.targets),
                                     y=train_loader.dataset.targets)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = FocalLoss(alpha=class_weights_tensor, gamma=2.0, reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=0.001)

patience = 5  # Number of epochs with no improvement before stopping
best_loss = float('inf')  # Initialize best loss to a very high value
epochs_without_improvement = 0  # Counter for patience



Using device: cuda


C:\Users\Korisnik\AppData\Local\Temp\ipykernel_32216\1101473969.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loader = torch.load('train_dataloader.pth')
C:\User

In [8]:
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}",
                               ncols=100):  # Assuming train_loader is defined
        images, labels = images.to(device), labels.to(device)

        # Zero gradients, perform a backward pass, and update weights
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss.item() * images.size(0)

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Step the learning rate scheduler
    scheduler.step()

    # Print statistics
    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples * 100

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    print(f"Learning Rate: {scheduler.get_last_lr()[0]:.6f}")  # Print the current learning rate


Epoch 1/50: 100%|█████████████████████████████████████████████████| 449/449 [05:45<00:00,  1.30it/s]


Epoch [1/50], Loss: 1.5300, Accuracy: 16.47%
Learning Rate: 0.001000


Epoch 2/50: 100%|█████████████████████████████████████████████████| 449/449 [00:30<00:00, 14.86it/s]


Epoch [2/50], Loss: 1.3221, Accuracy: 20.26%
Learning Rate: 0.001000


Epoch 3/50: 100%|█████████████████████████████████████████████████| 449/449 [00:26<00:00, 16.72it/s]


...

Epoch [48/50], Loss: 0.5042, Accuracy: 54.58%
Learning Rate: 0.000063


Epoch 49/50: 100%|████████████████████████████████████████████████| 449/449 [00:23<00:00, 18.87it/s]


Epoch [49/50], Loss: 0.5052, Accuracy: 54.70%
Learning Rate: 0.000063


Epoch 50/50: 100%|████████████████████████████████████████████████| 449/449 [00:24<00:00, 18.55it/s]

Epoch [50/50], Loss: 0.4926, Accuracy: 54.87%
Learning Rate: 0.000031


In [9]:
import torch
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Set model to evaluation mode
model.eval()

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']


# Containers for predictions and true labels
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Print classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=emotion_labels))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

Classification Report:
              precision    recall  f1-score   support

       Angry       0.43      0.42      0.43       958
     Disgust       0.70      0.52      0.60       111
        Fear       0.36      0.28      0.31      1024
       Happy       0.81      0.67      0.74      1774
         Sad       0.47      0.55      0.51      1233
    Surprise       0.40      0.50      0.44      1247
     Neutral       0.69      0.75      0.72       831

    accuracy                           0.54      7178
   macro avg       0.55      0.53      0.54      7178
weighted avg       0.55      0.54      0.54      7178

Confusion Matrix:
[[ 405   13  119   45  142  201   33]
 [  32   58    5    1    3   10    2]
 [ 151    3  286   44  141  260  139]
 [  96    0   64 1196  174  174   70]
 [  85    2   88   85  676  279   18]
 [ 154    6  130   60  261  619   17]
 [  19    1  100   39   30   16  626]]
